Este programa de python combina el cliente de TD con pytd y un query de expresiones regulares para obtener una LISTA ÚNICA de los caracteres especiales que no son parte de una palabra.

Este caso puede ser ocupado para realizar un exploratory data analysis sobre los casos excepcionales que pueden ser remplazados. Por ejemplo, en la validación de nombres, AGRUPANDO CASOS POR CARACTERES ESPECIALES.

In [1]:
import pytd
import pandas as pd
client = pytd.Client(
apikey='', #Insert your API key.
database='src',
default_engine='presto'
)

In [39]:
#This query gets a list of lists where we have all non-word characters
#This results gets all the patterns and all possible values.
res = client.query(
    """
    select distinct REGEXP_EXTRACT_ALL(nombre_cte, '[^a-zA-Z\s\á\é\í\ó\ú\Á\É\Í\Ó\Ú\ü\Ü\ñ\Ñ]') as other
    from nvdr_history
    order by other asc
    """,engine = 'presto'
)
df = pd.DataFrame(**res)
df.head()

,other
0,[]
1,"[, ]"
2,"[""]"
3,"["", ""]"
4,"["", "", "", ""]"


In [55]:
#In this part we are going to unpack the values only in one list
# This works for regular nested list with list comprehension.
# https://appdividend.com/2022/06/17/how-to-flatten-list-in-python/
flatten_list = [element for sublist in df['other'] for element in sublist]
Special_characters = list(set(flatten_list)) #Getting the unique values into a single list.
print('This is the new length of the special characters list',len(Special_characters))
Special_characters.sort()


This is the new length of the special characters list 36


In [58]:
#Here we have the special characters in a sorted way.
print(Special_characters)

['\x1a', '"', '#', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '@', '_', '`', '|', '}', 'Ƭ', '‡', '‹', '�']
